# ⚽ Modélisation et Analyse de Données Footballistiques

In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [59]:
# les matches : 
df_matches_from_csv = pd.read_csv('../data/data_cleaned/matches_clean.csv')
df_matches_from_csv.head(5)

,Squad,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp_Formation,Referee,MatchKey
0,Liverpool,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2,0,Ipswich Town,2.6,0.5,62.0,30014,Virgil van Dijk,4-2-3-1,4-2-3-1,Tim Robinson,2024-08-17 00:00:00_Ipswich Town_Liverpool
1,Liverpool,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,2.5,0.5,62.0,60017,Virgil van Dijk,4-2-3-1,4-4-2,Stuart Attwell,2024-08-25 00:00:00_Brentford_Liverpool
2,Liverpool,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3,0,Manchester Utd,1.8,1.4,47.0,73738,Virgil van Dijk,4-2-3-1,4-2-3-1,Anthony Taylor,2024-09-01 00:00:00_Liverpool_Manchester Utd
3,Liverpool,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,0.9,0.4,68.0,60344,Virgil van Dijk,4-2-3-1,4-2-3-1,Michael Oliver,2024-09-14 00:00:00_Liverpool_Nott'ham Forest
4,Liverpool,2024-09-17,21:00,Champions Lg,League phase,Tue,Away,W,3,1,it Milan,3.1,0.6,51.0,59826,Virgil van Dijk,4-2-3-1,4-2-3-1,Espen Eskås,2024-09-17 00:00:00_Liverpool_it Milan


In [60]:
# les matches : 
df_joueurs_from_csv = pd.read_csv('../data/data_cleaned/players_clean.csv')
df_joueurs_from_csv.head(5)

,Squad,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR
0,Liverpool,Mohamed Salah,eg EGY,FW,32.0,38,38,3371.0,37.5,29.0,18.0,47.0,20.0,9.0,9.0,1.0,0.0
1,Liverpool,Virgil van Dijk,nl NED,DF,33.0,37,37,3330.0,37.0,3.0,1.0,4.0,3.0,0.0,0.0,5.0,0.0
2,Liverpool,Ryan Gravenberch,nl NED,MF,22.0,37,37,3160.0,35.1,0.0,4.0,4.0,0.0,0.0,0.0,6.0,1.0
3,Liverpool,Alexis Mac Allister,ar ARG,MF,25.0,35,30,2599.0,28.9,5.0,5.0,10.0,5.0,0.0,0.0,6.0,0.0
4,Liverpool,Ibrahima Konaté,fr FRA,DF,25.0,31,30,2560.0,28.4,1.0,2.0,3.0,1.0,0.0,0.0,5.0,0.0


## Extraction des tables de base de données depuis un fichier CSV et Stocker dans un fichier CSV

###  🟢 Table saison(id_saison, annee) : 

In [61]:
list_saisons = ['2024-2025']
print(list_saisons)

print("Nombre total de saison :", len(list_saisons))


['2024-2025']
Nombre total de saison : 1


In [62]:

df_saisons = pd.DataFrame(list_saisons, columns=['Annees'])
df_saisons['Annees'] = '2024-2025'
df_saisons['id_saison'] = range(1, len(df_saisons) + 1)

df_saisons = df_saisons[['id_saison', 'Annees']]

df_saisons.head(5)

,id_saison,Annees
0,1,2024-2025


In [63]:
#  enregistrer dans les saisons dans un fichier CSV : 

df_saisons.to_csv('../data/data_load/data_saison.csv', index=False)


* extraire le id de saison pour l'utilisation dans les restes : 

In [64]:
id_saison = df_saisons.query("Annees == '2024-2025'")['id_saison'].values[0]
# print(id_saison)

***


###  🟢 Table equipe(idequipe, nomequipe, idcompetition, idsaison) : 

In [65]:

list_team_home = df_matches_from_csv['Squad'].unique().tolist()
list_team_away = df_matches_from_csv['Opponent'].unique().tolist()

print("Nombre d'équipes à domicile uniques :", len(list_team_home))
print("Nombre d'équipes à l'extérieur uniques :", len(list_team_away))

# Combiner les deux listes
list_teams = list_team_home + list_team_away

# Supprimer les doublons
list_teams = sorted(list(set(list_teams)))

# print(list_teams)

print("Nombre total d'équipes uniques :", len(list_teams))


Nombre d'équipes à domicile uniques : 20
Nombre d'équipes à l'extérieur uniques : 112
Nombre total d'équipes uniques : 113


In [66]:

df_teams = pd.DataFrame(list_teams, columns=['Equipe'])
df_teams['id_saison'] = id_saison
df_teams['id_equipe'] = range(1, len(df_teams) + 1)

# Réorganiser les colonnes dans l'ordre id_equipe => equipe => id_saison
df_teams = df_teams[['id_equipe', 'Equipe', 'id_saison']]

df_teams.head(5)

,id_equipe,Equipe,id_saison
0,1,AFC Wimbledon,1
1,2,Acc'ton Stanley,1
2,3,Arsenal,1
3,4,Aston Villa,1
4,5,Barnsley,1


In [67]:
#  enregistrer dans les equipe dans un fichier CSV : 

df_teams.to_csv('../data/data_load/data_equipes.csv', index=False)

***

###  🟢 Table competition(idcompetition, nomcompetition): 

In [68]:
print(f"Le nombre total des matches : {len(df_matches_from_csv)}")

# Récupérer toutes les compétitions depuis le df_mathes
list_com = df_matches_from_csv['Comp'].tolist()

print("\nAvant suppression des doublons : le nombre total des entrées est ====>", len(list_com))

# Supprimer les doublons
competitions = list(df_matches_from_csv['Comp'].unique())

print("\nAprès suppression des doublons : le nombre total des compétitions est ====>", len(competitions))
print(competitions)

Le nombre total des matches : 559

Avant suppression des doublons : le nombre total des entrées est ====> 559

Après suppression des doublons : le nombre total des compétitions est ====> 7
['Premier League', 'Champions Lg', 'EFL Cup', 'FA Cup', 'FA Community Shield', 'Conf Lg', 'Europa Lg']


In [69]:

df_competitions = pd.DataFrame(competitions, columns=['nom_competition'])
df_competitions['id_competition'] = range(1, len(df_competitions) + 1)

# Réorganiser les colonnes dans l'ordre id_competition => competition
df_competitions = df_competitions[['id_competition', 'nom_competition']]

df_competitions

,id_competition,nom_competition
0,1,Premier League
1,2,Champions Lg
2,3,EFL Cup
3,4,FA Cup
4,5,FA Community Shield
5,6,Conf Lg
6,7,Europa Lg


In [70]:
#  enregistrer dans les competitions dans un fichier CSV : 

df_competitions.to_csv('../data/data_load/data_competitions.csv', index=False)

###  🟢 Table match (idmatch_, date_match, heure, round, venue, idteamhome, idteam__away, id_competition, id_saison): 

In [71]:
# Récupérer les ids : 

equipe_to_id = dict(zip(df_teams['Equipe'], df_teams['id_equipe']))

comp_to_id = dict(zip(df_competitions['nom_competition'], df_competitions['id_competition']))

id_saison = df_saisons[df_saisons['Annees'] == '2024-2025']['id_saison'].values[0]

matches_data = []

for index, row in df_matches_from_csv.iterrows():
    
    idteamhome = equipe_to_id.get(row['Squad'])
    idteamaway = equipe_to_id.get(row['Opponent'])
    
    id_competition = comp_to_id.get(row['Comp'])

    
    # data_match
    data_matche = {
        'date_match': row['Date'],
        'heure': row['Time'],
        'round': row['Round'],
        'venue': row['Venue'],
        'id_team_home': idteamhome,
        'id_team_away': idteamaway,
        'id_competition': id_competition,
        'id_saison': id_saison
    }
    
    matches_data.append(data_matche)

# Créer le DataFrame df_matches
dataframe_matches = pd.DataFrame(matches_data)
dataframe_matches['id_match'] = range(1, len(dataframe_matches) + 1)
dataframe_matches = dataframe_matches[['id_match', 'date_match', 'heure', 'round', 'venue', 'id_team_home', 'id_team_away', 'id_competition', 'id_saison']]

print("Extrait des matches transformés :")
print(dataframe_matches.head(3))


Extrait des matches transformés :
   id_match  date_match  heure        round venue  id_team_home  id_team_away  \
0         1  2024-08-17  12:30  Matchweek 1  Away            30            27   
1         2  2024-08-25  16:30  Matchweek 2  Home            30            11   
2         3  2024-09-01  16:00  Matchweek 3  Away            30            33   

   id_competition  id_saison  
0               1          1  
1               1          1  
2               1          1  


In [72]:
#  enregistrer dans les competitions dans un fichier CSV : 

dataframe_matches.to_csv('../data/data_load/data_matches.csv', index=False)

###  🟢 resultatmatch(idresultat, idmatch, idequipe, butsmarques, butsconcedes, resultat ENUM('Victoire', 'Défaite', 'Nul')) : 

In [73]:
resultats_data = []

result_mapping = {
    'W': {'Home': 'Victoire', 'Away': 'Défaite', 'Neutral': 'Victoire'},
    'L': {'Home': 'Défaite', 'Away': 'Victoire', 'Neutral': 'Défaite'},
    'D': {'Home': 'Nul', 'Away': 'Nul', 'Neutral': 'Nul'}
}

def get_team_result(result_char, team_venue, match_venue):
    
    if match_venue == 'Neutral':
        return result_mapping[result_char]['Neutral']
    else:
        return result_mapping[result_char][team_venue]

team_id_to_name = dict(zip(df_teams['id_equipe'], df_teams['Equipe']))

for index, match_row in dataframe_matches.iterrows():
    idmatch = match_row['id_match']
    id_team_home = match_row['id_team_home']
    id_team_away = match_row['id_team_away']
    date_match = match_row['date_match']
    venue = match_row['venue']  # 'Home', 'Away' ou 'Neutral'
    
    # Récupérer les noms des équipes
    home_team_name = team_id_to_name.get(id_team_home)
    away_team_name = team_id_to_name.get(id_team_away)
    
    mask = (
        (df_matches_from_csv['Date'] == date_match) & 
        (
            ((df_matches_from_csv['Squad'] == home_team_name) & (df_matches_from_csv['Opponent'] == away_team_name)) |
            ((df_matches_from_csv['Squad'] == away_team_name) & (df_matches_from_csv['Opponent'] == home_team_name))
        )
    )
    
    matching_matches = df_matches_from_csv[mask]
    
    if len(matching_matches) == 0:
        print(f"❌ Match non trouvé: {date_match} - {home_team_name} vs {away_team_name}")
        continue
    
    original_match = matching_matches.iloc[0]
    
    if original_match['Squad'] == home_team_name:
        buts_marques_home = int(original_match['GF'])
        buts_marques_away = int(original_match['GA'])
        result_char = original_match['Result']
    else:
        buts_marques_home = int(original_match['GA'])
        buts_marques_away = int(original_match['GF'])
        result_char = original_match['Result']
    
    resultat_home = get_team_result(result_char, 'Home', venue)
    resultat_away = get_team_result(result_char, 'Away', venue)
    
    # Ajouter l'entrée pour l'équipe home
    resultats_data.append({
        'id_match': idmatch,
        'id_equipe': id_team_home,
        'buts_marques': buts_marques_home,
        'buts_concedes': buts_marques_away,
        'resultat': resultat_home
    })
    
    # Ajouter l'entrée pour l'équipe away
    resultats_data.append({
        'id_match': idmatch,
        'id_equipe': id_team_away,
        'buts_marques': buts_marques_away,
        'buts_concedes': buts_marques_home,
        'resultat': resultat_away
    })




In [74]:
# Créer le DataFrame final des résultats
df_resultats = pd.DataFrame(resultats_data)

df_resultats['id_resultat'] = range(1, len(df_resultats) + 1)

# Réorganiser les colonnes
colonnes_finales = ['id_resultat', 'id_match', 'id_equipe', 'buts_marques', 'buts_concedes', 'resultat']
df_resultats = df_resultats[colonnes_finales]

df_resultats.head(5)

,id_resultat,id_match,id_equipe,buts_marques,buts_concedes,resultat
0,1,1,30,2,0,Victoire
1,2,1,27,0,2,Défaite
2,3,2,30,2,0,Victoire
3,4,2,11,0,2,Défaite
4,5,3,30,3,0,Victoire


In [75]:
#  enregistrer dans les competitions dans un fichier CSV : 

df_resultats.to_csv('../data/data_load/data_resultat_match.csv', index=False)

###  🟢 joueur(id_joueur, nom_joueur, position, nationalite, id_equipe);

In [76]:
df_joueurs_from_csv.columns

Index(['Squad', 'Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s',
       'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR'],
      dtype='object')

In [77]:
# Récupérer les ids : 

joueurs_data = []

for index, row in df_joueurs_from_csv.iterrows():
    
    id_team = equipe_to_id.get(row['Squad'])
    
    # data_match
    data_joueur = {
        'nom_joueur': row['Player'],
        'position': row['Pos'],
        'nationalite': row['Nation'],
        'id_equipe': id_team,
    }
    
    joueurs_data.append(data_joueur)

# Créer le DataFrame df_matches
dataframe_players = pd.DataFrame(joueurs_data)
dataframe_players['id_joueur'] = range(1, len(dataframe_players) + 1)
dataframe_players = dataframe_players[['id_joueur', 'nom_joueur', 'position', 'nationalite', 'id_equipe']]

print("Extrait des matches transformés :")
print(dataframe_players.head(3))


Extrait des matches transformés :
   id_joueur        nom_joueur position nationalite  id_equipe
0          1     Mohamed Salah       FW      eg EGY         30
1          2   Virgil van Dijk       DF      nl NED         30
2          3  Ryan Gravenberch       MF      nl NED         30


In [78]:
#  enregistrer dans les competitions dans un fichier CSV : 

dataframe_players.to_csv('../data/data_load/data_players.csv', index=False)

###  🟢 statistiquejoueur(id_stats, id_joueur, buts, passes_decisives, nb_matches_played, cartons_jaunes, cartons_rouges);

In [79]:
df_joueurs_from_csv.columns

Index(['Squad', 'Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s',
       'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR'],
      dtype='object')

In [80]:
# id_stats, id_joueur, buts, passes_decisives, nb_matches_played, cartons_jaunes, cartons_rouges

# Récupérer les ids : 

player_to_id = dict(zip(dataframe_players['nom_joueur'], dataframe_players['id_joueur']))

print(player_to_id)

statistiques_joueurs_data = []

for index, row in df_joueurs_from_csv.iterrows():
    
    id_joueur = player_to_id.get(row['Player'])
    
    # data_match
    data_stats_player = {
        'id_joueur': id_joueur,
        'buts': row['Gls'],
        'passes_decisives': row['Ast'],
        'nb_matches_played': row['MP'],
        'cartons_jaunes': row['CrdY'],
        'cartons_rouges': row['CrdR'],
    }
    
    statistiques_joueurs_data.append(data_stats_player)

# Créer le DataFrame df_matches
dataframe_statistiques_joueurs = pd.DataFrame(statistiques_joueurs_data)
dataframe_statistiques_joueurs['id_stats'] = range(1, len(dataframe_statistiques_joueurs) + 1)
dataframe_statistiques_joueurs = dataframe_statistiques_joueurs[['id_stats', 'id_joueur', 'buts', 'passes_decisives', 'nb_matches_played', 'cartons_jaunes', 'cartons_rouges']]

print("Extrait des matches transformés :")
print(dataframe_statistiques_joueurs.head(3))


{'Mohamed Salah': 1, 'Virgil van Dijk': 2, 'Ryan Gravenberch': 3, 'Alexis Mac Allister': 4, 'Ibrahima Konaté': 5, 'Dominik Szoboszlai': 6, 'Andrew Robertson': 7, 'Alisson': 8, 'Luis Díaz': 9, 'Trent Alexander-Arnold': 10, 'Cody Gakpo': 11, 'Curtis Jones': 12, 'Diogo Jota': 13, 'Caoimhín Kelleher': 14, 'Kostas Tsimikas': 15, 'Darwin Núñez': 16, 'Conor Bradley': 17, 'Joe Gomez': 18, 'Jarell Quansah': 19, 'Harvey Elliott': 20, 'Wataru Endo': 21, 'Federico Chiesa': 22, 'Vitezslav Jaros': 23, 'Jayden Danns': 24, 'Harvey Davies': 25, 'James McConnell': 26, 'Tyler Morton': 27, 'Amara Nallo': 28, 'Trey Nyoni': 29, 'David Raya': 30, 'William Saliba': 31, 'Declan Rice': 32, 'Thomas Partey': 33, 'Leandro Trossard': 34, 'Gabriel Magalhães': 35, 'Jurriën Timber': 36, 'Martin Ødegaard': 37, 'Gabriel Martinelli': 38, 'Kai Havertz': 39, 'Bukayo Saka': 40, 'Mikel Merino': 41, 'Myles Lewis-Skelly': 42, 'Ben White': 43, 'Riccardo Calafiori': 44, 'Ethan Nwaneri': 45, 'Jakub Kiwior': 46, 'Jorginho': 47, 'R

In [81]:
#  enregistrer dans les competitions dans un fichier CSV : 

dataframe_statistiques_joueurs.to_csv('../data/data_load/data_statistiques_joueurs.csv', index=False)

In [82]:
print(dataframe_statistiques_joueurs.shape)
print(dataframe_players.shape)

(702, 7)
(702, 5)


In [83]:
dataframe_players_sorted = dataframe_players.sort_values(by='nom_joueur')
dataframe_players_sorted.to_csv('../data/data_load/data_statistiques_joueurs_sorted.csv', index=False)
dataframe_players_sorted

,id_joueur,nom_joueur,position,nationalite,id_equipe
132,133,Aaron Anselmino,DF,ar ARG,18
461,462,Aaron Cresswell,DF,eng ENG,55
340,341,Aaron Hickey,"DF,MF",sct SCO,11
667,668,Aaron Ramsdale,GK,eng ENG,45
65,66,Aaron Ramsdale,GK,eng ENG,3
...,...,...,...,...,...
404,405,Zachariah Marsh,FW,eng ENG,22
299,300,Zain Silcott-Duberry,FW,ms MSR,10
214,215,Álex Moreno,"DF,MF",es ESP,38
72,73,İlkay Gündoğan,MF,de GER,32
